# Génération des réseaux de formes à partir des règles
- Swim1 basé sur l'échantillon initial
 - Gén-1 : génération des formes d'après les contextes phonologiques
 - Gén-2 : génération du réseau d'après les contextes phonologiques
 - Filt-1 : extraction du sous-réseau symétrique
 - Filt-2 : génération du réseau non-orienté correspondant à Filt-1
 - Filt-3 : extraction des cliques maximales & fidèles
- Swim2 basé sur le réseau calculé par Swim1
 - Exp : génération d'un nouvel échantillon basé sur Swim1
 - Gén-1 : génération des formes sans contexte phonologique
 - Gén-2 : génération du réseau sans contexte phonologique
 - Filt-1 : extraction du sous-réseau symétrique
 - Filt-2 : génération du réseau non-orienté correspondant à Filt-1
 - Filt-3 : extraction des cliques maximales & fidèles
- Évaluation  

## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [1]:
# -*- coding: utf8 -*-
import codecs,operator,datetime,os,glob,cellbell
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [2]:
%matplotlib inline

In [3]:
import yaml

In [4]:
from ipywidgets import FloatProgress
from IPython.display import display, HTML

In [5]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [6]:
saut="\n"

### Préparation des matrices de traits

In [7]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

# Choix de l'échantillon et des règles
- *sampleFile* est le nom de l'échantillon de départ
- *analysisPrefix* est une partie du nom des règles

In [8]:
filePrefix="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/Longitudinal"
sampleFiles=glob.glob(filePrefix+"*.pkl")
def prefixEchantillon(numero,sampleType="",casesType=""):
    candidats=[]
    matchFile=ur"^.*/Longitudinal(-%s-T\d+-F\d+)%s\.pkl"%(numero,sampleType+casesType)
    for sample in sampleFiles:
        m=re.match(matchFile,sample)
        if m:
#            print (sample)
#            print (m.group(1))
            candidats.append(m.group(1))
    if len(candidats)==1:
        return candidats[0]
    else:
        print ("PB pas de nom unique correspondant",len(candidats))

In [9]:
goldPrefix="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/"
goldFile="MGC-171229-Verbes3.pkl"
sampleNumero="00"
sampleType="-X"
casesType="-Morphomes"
#sampleType=""
#casesType=""
sampleNumber=prefixEchantillon(sampleNumero,sampleType,casesType)
print (sampleNumber)
paperPrefix="-ISMo"
genFormeVotes=True
genCliques=True
listeFormesOutput=["FS","FP"]
genDigraphe=False
genGraphe=False
plotDistributionCliques=False
if casesType:
    samplePrefix=filePrefix+"%s-%s"%(sampleNumber,sampleType.strip("-"))+casesType
    tirageFile=samplePrefix+".pkl"
elif sampleType:
    samplePrefix=filePrefix+"%s-%s"%(sampleNumber,sampleType.strip("-"))
    tirageFile=filePrefix+sampleNumber+".pkl"
else:
    samplePrefix=filePrefix+"%s-X"%(sampleNumber)
    tirageFile=filePrefix+sampleNumber+".pkl"
sampleFile=samplePrefix+"-paradigmes.csv"
initialFile=filePrefix+prefixEchantillon(sampleNumero)+sampleType+"-paradigmes.csv"
print (initialFile)
analysisPrefix=samplePrefix
if debug: 
    logfile_name=analysisPrefix+"-network.log"
    logfile = codecs.open(logfile_name,mode='w',encoding="utf8")

-00-T10000-F3663
/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/Longitudinal-00-T10000-F3663-X-paradigmes.csv


In [10]:
#tirage=filePrefix+'-'+sampleNumber+'-Tirage-'+sampleType+'-170503-1907'+casesType+'.pkl'
#tirage=filePrefix+sampleNumber+casesType+'.pkl'
with open(tirageFile, 'rb') as input:
    sampleTirage = pickle.load(input)

In [57]:
if not "morphome" in sampleTirage.columns.tolist():
    sampleTirage["morphome"]=sampleTirage["case"]
morphomeCases=sampleTirage[["case","morphome"]].drop_duplicates().to_dict()
morphomeCases

{'case': {0: u'ppMS',
  1: u'pi2S',
  2: u'inf',
  4: u'pP',
  5: u'pi1P',
  6: u'ai3S',
  10: u'fi1P',
  16: u'pi1S',
  18: u'fi1S',
  19: u'ai1S',
  20: u'ii1S',
  25: u'pc1P',
  28: u'ps2S',
  32: u'ii2S',
  36: u'pi2P',
  49: u'fi2S',
  57: u'ai3P',
  70: u'ai1P',
  111: u'pI2S',
  114: u'pi3P',
  117: u'pc2P',
  134: u'pI2P',
  138: u'ii2P',
  320: u'ii1P'},
 'morphome': {0: u'ppMS',
  1: u'pi2S/pi3S',
  2: u'inf',
  4: u'pP',
  5: u'pi1P',
  6: u'ai3S/is3S',
  10: u'fi1P/fi3P',
  16: u'pi1S',
  18: u'fi1S/pc1S/pc2S/pc3P/pc3S',
  19: u'ai1S',
  20: u'ii1S/ii3P/ii3S',
  25: u'pc1P/ppFS',
  28: u'ps2S/ps3P/ps3S',
  32: u'ii2S/is1S/is3P/pI1P/ppMP/ps2P',
  36: u'pi2P',
  49: u'fi2S/fi3S',
  57: u'ai3P/fi2P',
  70: u'ai1P/ai2S/ppFP',
  111: u'pI2S',
  114: u'pi3P',
  117: u'pc2P',
  134: u'pI2P',
  138: u'ii2P',
  320: u'ii1P'}}

In [11]:
casesMC=morphomeCases["case"]
morphomesMC=morphomeCases["morphome"]
dictMorphomeCases={}
for element in casesMC:
    dictMorphomeCases[casesMC[element]]=morphomesMC[element].split("/")
dictMorphomeCases

{u'ai1P': [u'ai1P', u'ai2S', u'ppFP'],
 u'ai1S': [u'ai1S'],
 u'ai3P': [u'ai3P', u'fi2P'],
 u'ai3S': [u'ai3S', u'is3S'],
 u'fi1P': [u'fi1P', u'fi3P'],
 u'fi1S': [u'fi1S', u'pc1S', u'pc2S', u'pc3P', u'pc3S'],
 u'fi2S': [u'fi2S', u'fi3S'],
 u'ii1P': [u'ii1P'],
 u'ii1S': [u'ii1S', u'ii3P', u'ii3S'],
 u'ii2P': [u'ii2P'],
 u'ii2S': [u'ii2S', u'is1S', u'is3P', u'pI1P', u'ppMP', u'ps2P'],
 u'inf': [u'inf'],
 u'pI2P': [u'pI2P'],
 u'pI2S': [u'pI2S'],
 u'pP': [u'pP'],
 u'pc1P': [u'pc1P', u'ppFS'],
 u'pc2P': [u'pc2P'],
 u'pi1P': [u'pi1P'],
 u'pi1S': [u'pi1S'],
 u'pi2P': [u'pi2P'],
 u'pi2S': [u'pi2S', u'pi3S'],
 u'pi3P': [u'pi3P'],
 u'ppMS': [u'ppMS'],
 u'ps2S': [u'ps2S', u'ps3P', u'ps3S']}

# Préparation du calcul des analogies

### Calcul de la différence entre deux formes

In [12]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

### Accumulation des paires appartenant à un patron

In [13]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

### Transformation d'un patron en RegExp

In [14]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

### Substitution de sortie 
???

In [15]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

# Classe pour la gestion des patrons, des classes et des transformations

In [16]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            if debug: print ("le numéro de forme n'est pas dans [1,2]",n, file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
        self.classeCF={}
        self.nbClasseCF={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeFormeCF=[]
        regleFormeCF=""
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            filterF1=".*"+patron.split("-")[0]+"$"
            if re.match(filterF1,forme1):
                classeFormeCF.append(patron)
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleFormeCF=patron
            filterF1=self.patrons[patron]
            if re.match(filterF1,forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseFormeCF=", ".join(classeFormeCF)
        if not idClasseFormeCF in self.classeCF:
            self.classeCF[idClasseFormeCF]={}
            self.nbClasseCF[idClasseFormeCF]=0
        if not regleFormeCF in self.classeCF[idClasseFormeCF]:
            self.classeCF[idClasseFormeCF][regleFormeCF]=0
        self.nbClasseCF[idClasseFormeCF]+=1
        self.classeCF[idClasseFormeCF][regleFormeCF]+=1
        
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=True):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            if debug:
                print (forme, file=logfile) 
                print ("pas de patron", file=logfile)
        return sortieForme
        

## Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [17]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

### Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [18]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


# Lecture de l'échantillon

In [19]:
phonologicalMap=sampleType.strip("-")
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [20]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

### Vérification de la phonotactique des glides du français
- si *prononciation* est *None* renvoyer *None*
- ajout de diérèses dans les séquences mal-formées
- vérification des séquences consonne+glide à la finale

In [21]:
dierese={"j":"ij", "w":"uw","H":"yH","i":"ij","u":"uw","y":"yH"}

In [22]:
def checkFrench(prononciation):
    if prononciation:
        result=recoder(prononciation)
        m=re.match(ur"^.*([^ieèEaOouy926êôâ])[jwH]$",result)
        if m:
            print ("pb avec un glide final", prononciation)
        m=re.match(ur"(.*[ptkbdgfsSvzZ][rl])([jwH])(.*)",result)
        if m:
            n=re.search(ur"[ptkbdgfsSvzZ][rl](wa|Hi|wê)",result)
            if not n:
                glide=m.group(2)
                result=m.group(1)+dierese[glide]+m.group(3)
        m=re.match(ur"(.*)([iuy])([ieEaOouy].*)",result)
        if m:
            glide=m.group(2)
            result=m.group(1)+dierese[glide]+m.group(3)
        result=result.replace("Jj","J")
    else:
        result=prononciation
    return result

In [23]:
paradigmes=pd.read_csv(sampleFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]
paradigmes=paradigmes.dropna(axis=1,how='all')

In [24]:
goldCases=paradigmes.columns.tolist()
goldCases.remove("lexeme")
#goldCases

- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [25]:
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")
#sampleCases

In [26]:
paradigmes.stack().value_counts(dropna=True).sum()

4712

In [27]:
len(paradigmes.dropna(thresh=1)["lexeme"])

1289

## Préparation des données initiales et de référence

In [28]:
listeTest=paradigmes.dropna(thresh=1)["lexeme"].values.tolist()
#listeTest=[u"asseoir",u"balayer",u"manger"]
nbVerbes=len(listeTest)
#print (nbVerbes)

### Préparation des formes initiales

In [29]:
initialParadigmes=pd.read_csv(initialFile,sep=";",encoding="utf8")
del initialParadigmes[u"Unnamed: 0"]
initialParadigmes=initialParadigmes.dropna(axis=1,how='all')

In [30]:
initialForms=pd.melt(initialParadigmes[paradigmes["lexeme"].isin(listeTest)],id_vars=["lexeme"]).dropna()

initialForms["lexeme-case"]=initialForms["lexeme"]+"-"+initialForms["variable"]
initialForms.drop(labels=["lexeme","variable"],axis=1,inplace=True)

initialForms.set_index(["lexeme-case"],inplace=True)

initialFormsIndex=initialForms.index.tolist()

### Préparation des formes de référence

In [31]:
#with open(goldPrefix+"/"+goldFile,"rb") as input:
with open(goldPrefix+goldFile,"rb") as input:
    lexiqueGold=pickle.load(input)

'''Rectifications phonologiques'''
lexiqueGold["phono"]=lexiqueGold["phono"].apply(lambda x: checkFrench(x))
completeParadigmes=pd.pivot_table(lexiqueGold, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()

'''Mise en liste des formes de références'''
goldTestForms=pd.melt(completeParadigmes[completeParadigmes["lexeme"].isin(listeTest)],id_vars=["lexeme"]).dropna()
goldTestForms["lexeme-case"]=goldTestForms["lexeme"]+"-"+goldTestForms["case"]
goldTestForms.drop(labels=["lexeme","case"],axis=1,inplace=True)
goldTestForms.set_index(["lexeme-case"],inplace=True)

'''Extraction des formes de références pertinentes'''
goldForms=goldTestForms.loc[~goldTestForms.index.isin(initialFormsIndex)]
goldFormsIndex=goldForms.index.tolist()

# Lecture des règles

In [32]:
with open(analysisPrefix+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)

### Comparer les cases analysées avec l'ensemble de toutes les cases

In [33]:
analyseCases=list(set([case for (case,autre) in resultatsLecture.keys()]))
if sorted(analyseCases)!=sorted(goldCases):
    print ("Attention l'analyse ne comprend pas toutes les cases")
    print (sorted(analyseCases))
    print (sorted(goldCases))

# Préparations pour la génération des formes

In [34]:
class paradigmeDistribution:
    '''
    Gestion des distributions dans les cases du paradigme
    '''

    def __init__(self,lexeme):
        self.lexeme=lexeme
        self.formes={i:{} for i in analyseCases}

    def ajouterFormes(self,case,formes,coef=1.0):
        for forme in formes:
            if not forme in self.formes[case]:
                self.formes[case][forme]=0
            self.formes[case][forme]+=formes[forme]*coef
            
    def normaliserDistributions(self,caseListe=analyseCases):
        normalesDistributions={i:{} for i in caseListe}
        for case in caseListe:
            total=0
            for element in self.formes[case]:
                total+=self.formes[case][element]
            for element in self.formes[case]:
                normalesDistributions[case][element]=float(self.formes[case][element])/total
        return normalesDistributions
        

In [35]:
def generateForms(lexeme,contextFree=False):
    candidats=paradigmeDistribution(lexeme)
    casesSamples=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
    casesSamples.remove("lexeme")
    for caseDepart in casesSamples:
        formeDepart=paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]
        if debug: print (caseDepart,formeDepart, file=logfile)
#        if formeDepart!="nan":
        for case in analyseCases:
            if debug: print (case, file=logfile)
            if not isinstance(resultatsLecture[(caseDepart, case)],str):
                if "," in formeDepart:
                    formesDepart=formeDepart.split(",")
                    coef=1.0/len(formesDepart)
                    for element in formesDepart:
                        candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(element,contextFree),coef)
                else:
                    candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(formeDepart,contextFree))
            else: 
                if debug: print ("str", resultatsLecture[(caseDepart, case)], file=logfile)
    return candidats

In [36]:
def ajouterPoint(lexeme,forme,case,digraphe,graphe):
    pointName="%s-%s-%s"%(lexeme,forme,case)
#    if not pointName in digraphe.nodes():
    tam=case[:2]
    if tam=="in": tam="inf"
    digraphe.add_node(pointName, tam='"%s"'%tam)
    graphe.add_node(pointName, tam='"%s"'%tam)
    return pointName

def ajouterFleche(pointDepart,pointSortie,coef,digraphe,graphe):
    digraphe.add_edge(pointDepart,pointSortie,weight=float(coef))
    if digraphe.has_edge(pointSortie,pointDepart):
        coefGraphe=float(digraphe.edge[pointSortie][pointDepart]["weight"]+coef)/2
        graphe.add_edge(pointDepart,pointSortie,weight=coefGraphe)

In [37]:
def generateParadigms(generation1,genDigraphe=True,contextFree=False):
    lexeme=generation1.lexeme
    distributionInitiale=generation1.normaliserDistributions()
    candidats=paradigmeDistribution(lexeme)
    digraphe=nx.DiGraph()
    graphe=nx.Graph()    
    for caseDepart in analyseCases:
        for formeDepart in distributionInitiale[caseDepart]:
            if formeDepart:
                pointDepart=ajouterPoint(lexeme,formeDepart,caseDepart,digraphe,graphe)
                coefDepart=distributionInitiale[caseDepart][formeDepart]
                if debug: print (caseDepart,formeDepart, file=logfile)
                for caseSortie in analyseCases:
                    distributionSortieBrute=resultatsLecture[(caseDepart, caseSortie)].sortirForme(formeDepart,contextFree)
                    if distributionSortieBrute:
                        if not genDigraphe:
#                            print ("brute",distributionSortieBrute)
                            distributionSortie={f:distributionSortieBrute[f] for f in distributionSortieBrute if f in distributionInitiale[caseSortie]}
                        else:
                            distributionSortie=distributionSortieBrute
#                        print ("filtre",distributionSortie)
#                        print (distributionInitiale[caseSortie])
                        if debug: print (caseSortie,distributionSortie,distributionInitiale[caseDepart], file=logfile)
                        candidats.ajouterFormes(caseSortie,distributionSortie,distributionInitiale[caseDepart][formeDepart])
                        for formeSortie in distributionSortie:
                            pointSortie=ajouterPoint(lexeme,formeSortie,caseSortie,digraphe,graphe)
                            coefSortie=distributionSortie[formeSortie]
                            ajouterFleche(pointDepart,pointSortie,float(coefDepart*coefSortie),digraphe,graphe)
    return (candidats,digraphe,graphe)

# Génération d'un jeu de cliques

In [38]:
def generate(lexeme,genDigraphe=True,contextFree=False):
#    print (lexeme,end=", ")
    generation1=generateForms(lexeme,contextFree)
#    print ("génération 2",end=", ")
    (generation2,lexDigraphe,lexGraphe)=generateParadigms(generation1,genDigraphe,contextFree)
    lexCliques=list(nx.algorithms.clique.find_cliques(lexGraphe))
#    print (lexCliques)
#    print ("génération 3")
    return (generation2,lexDigraphe,lexGraphe,lexCliques)

In [39]:
paradigmes.dropna(thresh=1).count().sum()-paradigmes.dropna(thresh=1)["lexeme"].count()

3423

#### Calculer le score de la clique

In [40]:
def cliqueScore(clique,graph):
    score=0
    if len(clique)>1:
        for (depart,arrivee) in it.combinations_with_replacement(clique,2):
            score+=graph[depart][arrivee]["weight"]
    return score

In [41]:
def splitArrivee(arrivee):
    arriveeMorceaux=arrivee.split("-")
    if len(arriveeMorceaux)<3:
        print (arrivee,arriveeMorceaux)
    lexeme="-".join(arriveeMorceaux[:-2])
    formeArrivee=arriveeMorceaux[-2]
    caseArrivee=arriveeMorceaux[-1]
    return (lexeme,formeArrivee,caseArrivee)
    
# trouver tous les liens vers FS-* et FP-*
# regrouper par forme 
# calculer les proportions
# renvoyer les proportions par forme
# avec le nombre de forme à l'appui
def formeScore(forme,graph):
    scores={}
    scoresNormes={}
    for depart in graph.edge:
        for arrivee in graph.edge[depart]:
            (lexeme, formeArrivee, caseArrivee)=splitArrivee(arrivee)
            if caseArrivee==forme:
#                print (depart, formeArrivee, graph.edge[depart][arrivee])
                if not formeArrivee in scores:
                    scores[formeArrivee]=0
                scores[formeArrivee]+=graph.edge[depart][arrivee]["weight"]
    totalArrivee=0
    for formeArrivee in scores:
        totalArrivee+=scores[formeArrivee]
    for formeArrivee in scores:
        scoresNormes[formeArrivee]=scores[formeArrivee]/totalArrivee
    return (scores,scoresNormes)
        

## Préparations pour SWIM

In [42]:
def generateAnalysis(globDigraphe,globGraphe,contextFree=False):
    numClique=0
    progressBar = FloatProgress(min=0, max=nbVerbes-1,description="Generation (%d verbs)"%nbVerbes)
    display(progressBar)
    for i,element in enumerate(listeTest):
        cliquesScores[element]={}
        cliquesListes[element]={}
    #    if (i%100)==0: print (i, dateheure()[-4:], int(100*float(i)/nbVerbes), end=", ")
        progressBar.value=i
        #print (element)
        result=generate(element,genDigraphe,contextFree)
        (generation,lexDigraphe,lexGraphe,lexCliques)= result
    #    print (generation,lexDigraphe,lexGraphe,lexCliques)
        if genFormeVotes:
            formesScores[element]={}
            formesScoresNormes[element]={}
            for formeOutput in listeFormesOutput:
                (formesScores[element][formeOutput],formesScoresNormes[element][formeOutput])=formeScore(formeOutput,lexDigraphe)
        if genDigraphe:
            globDigraphe=nx.union(globDigraphe,lexDigraphe)
        if genGraphe:
            globGraphe=nx.union(globGraphe,lexGraphe)
        cliques.extend(lexCliques)
        for clique in lexCliques:
            cliquesScores[element][numClique]=cliqueScore(clique,lexGraphe)
            cliquesListes[element][numClique]=clique
            numClique+=1
    return globDigraphe,globGraphe,numClique

In [43]:
def dictCliqueForms(clique):
    result={}
    for element in clique:
        lexeme,forme,case=splitArrivee(element)
        for c in dictMorphomeCases[case]:
            result[c]=forme
    return result

def dictPdRowForms(row):
    result={}
    for case in sampleCases:
        print (case,row[case].values[0])
    return result

def tableZero(case):
    if case in sampleCases:
        return u"Ø"
    else:
        return u"="

def makeTable(dictForms,title=""):
    tabular=[]
    labelTenseCode={"pi":"Present","ii":"Imperfective","ai":"Simple Past","fi":"Future",
                    "ps":"Subjunctive Pres.","is":"Subjunctive Imp.","pc":"Conditional","pI":"Imperative",
                    "inf":"Infinitive",
                    "ppMS":"Past Part. MS","ppMP":"Past Part. MP",
                    "ppFS":"Past Part. FS","ppMP":"Past Part. FP"
                   }
    def makeLine6(tenseCode):
        line=[]
        line.append(r"<th>%s</th>"%labelTenseCode[tenseCode])
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            case=tenseCode+person
            if (case in dictForms) and (not (type(dictForms[case]) == float and np.isnan(dictForms[case]))):
                line.append(r"<td>%s</td>"%(dictForms[case]))
            else:
                line.append(r"<td>%s</td>"%(tableZero(case)))
        return r"<tr>"+r"".join(line)+r"</tr>"

    def makeLine3(tenseCode):
        line=[]
        line.append(r"<th>%s</th>"%labelTenseCode[tenseCode])
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            if person in ["2S","1P","2P"]:
                case=tenseCode+person
                if case in dictForms and (not (type(dictForms[case]) == float and np.isnan(dictForms[case]))):
                    line.append(r"<td>%s</td>"%(dictForms[case]))
                else:
                    line.append(r"<td>%s</td>"%(tableZero(case)))
            else:
                line.append(r"<td>%s</td>"%(u"---"))
        return r"<tr>"+r"".join(line)+r"</tr>"
    
    def makeLineNF():
        line=[]
        line.append(r"<th>%s</th>"%"NF")
        for case in ["inf","pP","ppMS","ppMP","ppFS","ppFP"]:
            if case in dictForms and (not (type(dictForms[case]) == float and np.isnan(dictForms[case]))):
                line.append(r"<td>%s</td>"%(dictForms[case]))
            else:
                line.append(r"<td>%s</td>"%(tableZero(case)))
        return r"<tr>"+r"".join(line)+r"</tr>"
    
        
    top=[
        r"<table>",
        r"<caption style='caption-side:bottom;text-align:center'>",
        "Verbe : %s"%title,
        r"</caption>",
#        r"<tr><th/><th>1S</th><th>2S</th><th>3S</th><th>1P</th><th>2P</th><th>3P</th></tr>"
        r"<tr><th/><th>1SG</th><th>2SG</th><th>3SG</th><th>1PL</th><th>2PL</th><th>3PL</th></tr>"
        ]
    bottom=[
        r"</table>"
        ]
    tabular.append("\n".join(top))
    for tenseCode in ["pi","ii","fi","pc", "ps","ai", "is"]:
        tabular.append(makeLine6(tenseCode))
    tabular.append(makeLine3("pI"))
    tabular.append(makeLineNF())
    tabular.append("\n".join(bottom))
    return "\n".join(tabular)    

def diffParadigme(lexeme):
    outLen=lexemeMaxCliques[lexeme][0]
    inLen=paradigmes[paradigmes["lexeme"]==lexeme].notnull().sum(axis=1).values[0]-1
    if outLen>inLen:
        print (lexemeMaxCliques[lexeme][1])
        print (paradigmes[paradigmes["lexeme"]=="grandir"].values)
    return outLen-inLen
    

In [44]:
def checkFidelite(fidelite,clique):
    lFidele=False
    for element in clique:
        if fidelite in element:
            lFidele=True
    return lFidele

def generateCliques(contextFree=False):
    
    def bruteCliques(lexeme,maxCliqueSize=51):
        cliquesBrutes={n+1:0 for n in range(maxCliqueSize)}
        for l in cliquesListes[lexeme].values():
            longueur=len(l)
            if longueur>1:
                if not longueur in cliquesBrutes:
                    cliquesBrutes[longueur]=0
                cliquesBrutes[longueur]+=1
        return cliquesBrutes
    
    globDigraphe=nx.DiGraph()
    globGraphe=nx.Graph()

    globDigraphe,globGraphe,numClique=generateAnalysis(globDigraphe,globGraphe,contextFree)
    print 

    lexemeMaxCliques={}
    lexemeParadigmes={}
    progressBarCliques = FloatProgress(min=0, max=len(cliquesListes)-1,description="Analysis (%d verbes)"%len(cliquesListes))
    display(progressBarCliques)
    for lexeme in cliquesListes:
        progressBarCliques.value+=1
        maxLen=max([len(c) for c in cliquesListes[lexeme].values()])
        lexemeMaxCliques[lexeme]=bruteCliques(lexeme,maxLen)
        print (lexeme,"Nombre de cliques",sum([v for k,v in lexemeMaxCliques[lexeme].iteritems()]))
        maxNbCliques=max([v for k,v in lexemeMaxCliques[lexeme].iteritems()])
        if plotDistributionCliques:
            ax=pd.DataFrame.from_dict(lexemeMaxCliques[lexeme],orient="index").plot(kind="bar",legend=False,grid=True,figsize=(10,3))
            ax.set(xlim=(0,maxLen+.5),ylim=(0,maxNbCliques+10))
            ax.set_xlabel("Clique Size in Cells",fontsize=16)
            ax.set_ylabel("Number of Cliques",fontsize=16)

        dictParadigmes=paradigmes.set_index("lexeme").to_dict(orient="index")

        cliquesFideles={}
        fidelites=[v+"-"+k for k,v in dictParadigmes[lexeme].iteritems() if isinstance(v,unicode)]
        for l in cliquesListes[lexeme].values():
            longueur=len(l)
            if longueur>1:
                fidele=True
                for fidelite in fidelites:
                    if "," in fidelite:
                        fideliteForme,fideliteCase=fidelite.split("-")
                        fideliteFormes=fideliteForme.split(",")
                        fideliteItems=[fideliteF+"-"+fideliteCase for fideliteF in fideliteFormes]
#                        print (fideliteItems)
                        lFidele=True
                        for f in fideliteItems:
                            lFidele=lFidele & checkFidelite(f,l)
                    else:
                        lFidele=checkFidelite(fidelite,l)
                    if not lFidele:
                        fidele=False
                        break
                if fidele:
                    if not longueur in cliquesFideles:
                        cliquesFideles[longueur]=[]
                    cliquesFideles[longueur].append(l)
#                else:
#                    if lexeme==u"bégayer": print ()
#        print ([(k,len(v)) for k,v in cliquesFideles.iteritems()])
        if cliquesFideles:
            maxCliquesCard=max([k for k,v in cliquesFideles.iteritems()])
    #        print (maxCliquesCard)
    #        print (cliquesScores[lexeme])
    #        print (cliquesListes[lexeme])
            lexemeParadigmes[lexeme]=[]
            maxScoreCliques=max([clique for cliqueNumber, clique in cliquesScores[lexeme].items()])
            maxCardScoreNums=[numC for numC, c in cliquesListes[lexeme].items() if c in cliquesFideles[maxCliquesCard]]
            maxCardScore=max([scoreC for numC, scoreC in cliquesScores[lexeme].items() if numC in maxCardScoreNums])
    #        print ("max score among all cliques:",maxScoreCliques)
            print ("max score among faithfull cliques of %d forms:"%maxCliquesCard,maxCardScore)
            for c in cliquesFideles[maxCliquesCard]:
                cNumber=[cliqueNumber for cliqueNumber, clique in cliquesListes[lexeme].items() if clique == c]
                if len(cNumber)!=1:
                    print ("TOO MANY SCORES PROBLEM WITH CLIQUE", cNumber)
    #            print ("Liste n°",cNumber[0],cliquesScores[lexeme][cNumber[0]])
    #            print (sorted(cliquesScores[lexeme].items(), key=operator.itemgetter(1)))
    #            display(HTML(makeTable(dictCliqueForms(c),title=c[0].split("-")[0])))
    #            print (cliquesScores[lexeme][cNumber[0]], maxCardScore)
                if cliquesScores[lexeme][cNumber[0]]==maxCardScore:
                    lexemeParadigmes[lexeme].append(c)
        else:
            lexemeParadigmes[lexeme]=[[lexeme+"-"+f for f in fidelites]]
            print (u"no new faithfull clique, the previous one contained %d forms"%len(lexemeParadigmes[lexeme][0]))
    return lexemeParadigmes

In [45]:
def cutNodeName(nodeName):
    items=nodeName.split("-")
    nbItems=len(items)
    if nbItems>3:
        items=["-".join(items[0:nbItems-2]),items[-2],items[-1]]
    return items

def filledOutClique(cliques):
    if cliques:
        result=[]
        for clique in cliques:
#            print (clique)
            fullClique=[]
            for element in clique:
                if debug: print ("element",element)
                lexeme,forme,case=splitArrivee(element)
                for c in dictMorphomeCases[case]:
                    fullClique.append("-".join([lexeme,forme,c]))
            result.append(fullClique)
        return result
    else:
        return cliques

In [46]:
def extendParadigmes(contextParadigmes,extendMorphomes=False):
    lexemesParadigmeListe=[]
    for lexeme in contextParadigmes:
        if extendMorphomes:
            lexParadigmes=filledOutClique(contextParadigmes[lexeme])
        else:
            lexParadigmes=contextParadigmes[lexeme]
        if len(lexParadigmes)!=1:
            if debug:
                print ("LEXEME WITH A NON UNIQUE PARADIGM PB",len(lexParadigmes),lexeme)
                print (lexParadigmes)
        lexParadigme=lexParadigmes[0]
        for lexForme in lexParadigme:
            lexemesParadigmeListe.append(cutNodeName(lexForme))
    newForms=pd.DataFrame(lexemesParadigmeListe)
    newForms.columns=["lexeme","form","case"]
#    newParadigmes=newForms.pivot(index="lexeme", columns="case", values="form")
    newParadigmes=pd.pivot_table(newForms, values='form', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
    for i in newParadigmes.itertuples():
#        print (i[0],i[1])
        lexeme=i[1]
        lexemeIndexes=paradigmes.lexeme[paradigmes.lexeme==lexeme].index.tolist()
        if lexemeIndexes:
            lexemeIndex=lexemeIndexes[0]
        else:
            print (i,lexeme,lexemeIndexes)
        newParadigmes.loc[newParadigmes.lexeme==lexeme,"index"]=int(lexemeIndex)
    newParadigmes.set_index("index",inplace=True)   
    return paradigmes.combine_first(newParadigmes)

In [47]:
def countSplits(dfForms):
    dfForms.loc[:,"split"]=dfForms.loc[:,"value"].str.split(",")
    return dfForms["split"].str.len().sum()

def calculerResultats(contextParadigmes,extension="-Swim2"):
    
    '''Préparer le paradigme des prédictions'''
    brutParadigmes=extendParadigmes(contextParadigmes,extendMorphomes=False)
    finalParadigmes=extendParadigmes(contextParadigmes,extendMorphomes=True)
    finalParadigmes.to_csv(path_or_buf=analysisPrefix+"-paradigmes%s.csv"%extension,encoding="utf8",sep=";")
    finalTestForms=pd.melt(finalParadigmes[finalParadigmes["lexeme"].isin(listeTest)],id_vars=["lexeme"]).dropna()
    finalTestForms["lexeme-case"]=finalTestForms["lexeme"]+"-"+finalTestForms["variable"]
    finalTestForms.drop(labels=["lexeme","variable"],axis=1,inplace=True)
    finalTestForms.set_index(["lexeme-case"],inplace=True)
    
    '''Soustraire les formes initiales'''
    finalForms=finalTestForms.loc[~finalTestForms.index.isin(initialFormsIndex)]
    finalFormsIndex=finalForms.index.tolist()
    
    '''Calculer les sur/sous-générations'''
    underGeneration=goldForms.loc[~goldForms.index.isin(finalFormsIndex)]
    overGeneration=finalForms.loc[~finalForms.index.isin(goldFormsIndex)]
    
    '''Réduire les prédictions et la référence aux cases communes'''
    predictedForms=finalForms.loc[finalForms.index.isin(goldFormsIndex)]
    actualForms=goldForms.loc[goldForms.index.isin(finalFormsIndex)]
    
    '''Créer un tableau pour les comparaisons'''
    compareForms=predictedForms.copy()
    compareForms.loc[:,"right"]=actualForms.loc[:,"value"]
    
    '''Séparer les cases identiques des cases différentes'''
    sameForms=compareForms[compareForms["value"]==compareForms["right"]]
    diffForms=compareForms[compareForms["value"]!=compareForms["right"]]

    
    '''Sauvegarder les comparatifs'''
    overGeneration.to_csv(path_or_buf=analysisPrefix+"-overGeneration%s.csv"%extension,encoding="utf8")
    underGeneration.to_csv(path_or_buf=analysisPrefix+"-underGeneration%s.csv"%extension,encoding="utf8")
    sameForms.to_csv(path_or_buf=analysisPrefix+"-sameForms%s.csv"%extension,encoding="utf8")
    diffForms.to_csv(path_or_buf=analysisPrefix+"-diffForms%s.csv"%extension,encoding="utf8")
    
    
    '''Transformer les surabondances en liste'''
    diffForms.loc[:,"split-value"]=diffForms.loc[:,"value"].str.split(",")
    diffForms.loc[:,"split-right"]=diffForms.loc[:,"right"].str.split(",")

    '''Transformer les surabondances en set()'''
    diffForms.loc[:,"split-value"]=diffForms.loc[:,"split-value"].apply(set)
    diffForms.loc[:,"split-right"]=diffForms.loc[:,"split-right"].apply(set)
    
    '''Calculer le nombre de formes (y compris surabondances)'''
    nbValues=diffForms["split-value"].str.len().sum()
    nbRights=diffForms["split-right"].str.len().sum()

    '''Calculer les identités et les inclusions'''
    nbIdenticalSets=diffForms[diffForms["split-value"]==diffForms["split-right"]]["split-value"].str.len().sum()
    nbIncludedSets=diffForms[diffForms["split-value"]<diffForms["split-right"]]["split-value"].str.len().sum()
    nbWrongForms=(nbValues-nbIdenticalSets-nbIncludedSets)
    underBonus=(nbRights-nbIdenticalSets-nbIncludedSets)

    UG=countSplits(underGeneration)+underBonus
    OG=countSplits(overGeneration)
    TP=countSplits(sameForms)+nbIdenticalSets+nbIncludedSets
    FP=nbWrongForms
    resultCharacteristics=(UG,OG,TP,FP)
    recall=float(TP)/(UG+TP+FP)
    precision=float(TP)/(OG+TP+FP)
    fMeasure=2*recall*precision/(recall+precision)
    resultMeasures=(precision,recall,fMeasure)
    print ("UG",UG ,"OG",OG,"TP",TP,"FP",FP)
    print ("recall", recall, "precision", precision)
    print (fMeasure)
    return (brutParadigmes,finalParadigmes,resultCharacteristics,resultMeasures)


# SWIM1

In [48]:
cliques=[]
cliquesScores={}
cliquesListes={}

formesScores={}
formesScoresNormes={}

print (datetime.datetime.now())
%time swim1ContextParadigmes=generateCliques()

2018-01-02 11:36:25.994213


reprendre Nombre de cliques 42
max score among faithfull cliques of 8 forms: 33.7069160998
tâcher Nombre de cliques 2
max score among faithfull cliques of 7 forms: 27.8333333333
escamoter Nombre de cliques 11
max score among faithfull cliques of 11 forms: 65.0349737981
balayer Nombre de cliques 8
max score among faithfull cliques of 5 forms: 9.82380952381
surprendre Nombre de cliques 29
max score among faithfull cliques of 8 forms: 33.3822751323
rentrer Nombre de cliques 15
max score among faithfull cliques of 14 forms: 103.143037518
pêcher Nombre de cliques 15
max score among faithfull cliques of 11 forms: 65.25
dégotter Nombre de cliques 14
max score among faithfull cliques of 10 forms: 53.8001253133
assembler Nombre de cliques 3
max score among faithfull cliques of 8 forms: 35.75
amplifier Nombre de cliques 6
max score among faithfull cliques of 6 forms: 21.0
punir Nombre de cliques 21
max score among faithfull cliques of 3 forms: 5.22222222222
projeter Nombre de cliques 23
max scor

pulvériser Nombre de cliques 1
max score among faithfull cliques of 4 forms: 10.0
fonctionner Nombre de cliques 15
max score among faithfull cliques of 12 forms: 74.4911111111
paraître Nombre de cliques 39
max score among faithfull cliques of 4 forms: 7.95061728395
agrandir Nombre de cliques 17
max score among faithfull cliques of 7 forms: 25.6948051948
frotter Nombre de cliques 33
max score among faithfull cliques of 11 forms: 64.5223475355
révéler Nombre de cliques 22
max score among faithfull cliques of 9 forms: 40.6071428571
revenir Nombre de cliques 34
max score among faithfull cliques of 13 forms: 86.2262820513
éclater Nombre de cliques 15
max score among faithfull cliques of 13 forms: 88.3092520225
accomplir Nombre de cliques 11
max score among faithfull cliques of 3 forms: 4.75757575758
animer Nombre de cliques 8
max score among faithfull cliques of 12 forms: 77.3576923077
inciter Nombre de cliques 12
max score among faithfull cliques of 13 forms: 89.3896512373
enfiler Nombre d

remercier Nombre de cliques 25
max score among faithfull cliques of 4 forms: 10.0
réintégrer Nombre de cliques 2
max score among faithfull cliques of 9 forms: 44.45
matérialiser Nombre de cliques 26
max score among faithfull cliques of 12 forms: 73.8791208791
rugir Nombre de cliques 4
max score among faithfull cliques of 4 forms: 9.86363636364
diffracter Nombre de cliques 2
max score among faithfull cliques of 11 forms: 65.75
menacer Nombre de cliques 18
max score among faithfull cliques of 13 forms: 86.5229691361
commencer Nombre de cliques 17
max score among faithfull cliques of 8 forms: 34.8577694236
redevenir Nombre de cliques 25
max score among faithfull cliques of 9 forms: 38.8866666667
lutter Nombre de cliques 13
max score among faithfull cliques of 12 forms: 76.5833333333
envier Nombre de cliques 12
max score among faithfull cliques of 5 forms: 12.5227272727
résoudre Nombre de cliques 10
max score among faithfull cliques of 2 forms: 3.0
finir Nombre de cliques 35
max score amon

défiler Nombre de cliques 17
max score among faithfull cliques of 11 forms: 65.2321428571
courber Nombre de cliques 1
max score among faithfull cliques of 4 forms: 10.0
profiter Nombre de cliques 21
max score among faithfull cliques of 13 forms: 86.2377169143
bouillir Nombre de cliques 11
max score among faithfull cliques of 2 forms: 3.0
épeler Nombre de cliques 13
max score among faithfull cliques of 10 forms: 52.0078132652
avancer Nombre de cliques 16
max score among faithfull cliques of 10 forms: 51.8287907268
égoutter Nombre de cliques 9
max score among faithfull cliques of 11 forms: 64.8001253133
outrer Nombre de cliques 12
max score among faithfull cliques of 10 forms: 50.3535353535
gigoter Nombre de cliques 30
max score among faithfull cliques of 9 forms: 43.4583333333
prélever Nombre de cliques 13
max score among faithfull cliques of 10 forms: 54.1547619048
moisir Nombre de cliques 13
max score among faithfull cliques of 5 forms: 14.7207792208
réduire Nombre de cliques 14
max s

...

max score among faithfull cliques of 11 forms: 60.5036842105
distribuer Nombre de cliques 8
max score among faithfull cliques of 2 forms: 3.0
dégrader Nombre de cliques 16
max score among faithfull cliques of 13 forms: 89.2193172325
siffloter Nombre de cliques 12
max score among faithfull cliques of 11 forms: 65.6666666667
raconter Nombre de cliques 17
max score among faithfull cliques of 10 forms: 53.0746867168
fâcher Nombre de cliques 9
max score among faithfull cliques of 11 forms: 65.25
déterminer Nombre de cliques 11
max score among faithfull cliques of 12 forms: 77.3576923077
apparaître Nombre de cliques 53
max score among faithfull cliques of 7 forms: 25.2380952381
péter Nombre de cliques 42
max score among faithfull cliques of 13 forms: 78.3632478632
emballer Nombre de cliques 10
max score among faithfull cliques of 11 forms: 65.2321428571
enguirlander Nombre de cliques 11
max score among faithfull cliques of 12 forms: 76.25
muter Nombre de cliques 4
max score among faithfull c

mouler Nombre de cliques 11
max score among faithfull cliques of 6 forms: 20.1875
remarquer Nombre de cliques 15
max score among faithfull cliques of 9 forms: 43.5
saisir Nombre de cliques 20
max score among faithfull cliques of 4 forms: 9.42857142857
crépir Nombre de cliques 2
max score among faithfull cliques of 3 forms: 5.75
implorer Nombre de cliques 25
max score among faithfull cliques of 10 forms: 46.7333333333
immatriculer Nombre de cliques 5
max score among faithfull cliques of 9 forms: 44.4821428571
former Nombre de cliques 12
max score among faithfull cliques of 11 forms: 63.3080701754
brûler Nombre de cliques 16
max score among faithfull cliques of 11 forms: 64.1383928571
cesser Nombre de cliques 24
max score among faithfull cliques of 12 forms: 74.6336875158
presser Nombre de cliques 24
max score among faithfull cliques of 14 forms: 97.9607744337
canoniser Nombre de cliques 12
max score among faithfull cliques of 13 forms: 85.9473026973
retenir Nombre de cliques 40
max scor

bouffer Nombre de cliques 20
max score among faithfull cliques of 13 forms: 87.0627515759
latter Nombre de cliques 11
max score among faithfull cliques of 13 forms: 89.7987796517
charrier Nombre de cliques 10
max score among faithfull cliques of 6 forms: 20.1578947368
piétiner Nombre de cliques 5
max score among faithfull cliques of 12 forms: 72.38
distancer Nombre de cliques 11
max score among faithfull cliques of 12 forms: 75.1892344498
contenir Nombre de cliques 2
max score among faithfull cliques of 5 forms: 14.5833333333
élever Nombre de cliques 35
max score among faithfull cliques of 5 forms: 14.7
capturer Nombre de cliques 11
max score among faithfull cliques of 12 forms: 76.5852785924
apaiser Nombre de cliques 57
max score among faithfull cliques of 10 forms: 52.5799145299
barboter Nombre de cliques 25
max score among faithfull cliques of 11 forms: 61.303030303
asseoir Nombre de cliques 36
max score among faithfull cliques of 14 forms: 84.5363553114
entourer Nombre de cliques 6

manifester Nombre de cliques 5
max score among faithfull cliques of 12 forms: 77.625
intervenir Nombre de cliques 2
max score among faithfull cliques of 7 forms: 27.17
merder Nombre de cliques 41
max score among faithfull cliques of 10 forms: 54.2383333333
ronfler Nombre de cliques 5
max score among faithfull cliques of 8 forms: 35.75
maîtriser Nombre de cliques 14
max score among faithfull cliques of 14 forms: 102.986198696
bosser Nombre de cliques 40
max score among faithfull cliques of 11 forms: 65.4722222222
démerder Nombre de cliques 45
max score among faithfull cliques of 10 forms: 53.2121428571
tendre Nombre de cliques 49
max score among faithfull cliques of 6 forms: 16.8991798942
dépêcher Nombre de cliques 16
max score among faithfull cliques of 11 forms: 65.25
avoir Nombre de cliques 19
max score among faithfull cliques of 19 forms: 181.06871345
monter Nombre de cliques 17
max score among faithfull cliques of 11 forms: 64.333457611
porter Nombre de cliques 28
max score among f

reconstituer Nombre de cliques 11
max score among faithfull cliques of 5 forms: 14.0
lancer Nombre de cliques 12
max score among faithfull cliques of 10 forms: 54.2738095238
grimper Nombre de cliques 6
max score among faithfull cliques of 12 forms: 76.3615288221
griffer Nombre de cliques 15
max score among faithfull cliques of 13 forms: 86.0819589296
guérir Nombre de cliques 10
max score among faithfull cliques of 5 forms: 11.3122683052
daigner Nombre de cliques 11
max score among faithfull cliques of 11 forms: 65.5833333333
supprimer Nombre de cliques 12
max score among faithfull cliques of 12 forms: 75.7689882896
embrasser Nombre de cliques 15
max score among faithfull cliques of 14 forms: 102.046746382
bousculer Nombre de cliques 6
max score among faithfull cliques of 9 forms: 44.4821428571
recoucher Nombre de cliques 14
max score among faithfull cliques of 10 forms: 54.4166666667
endiguer Nombre de cliques 12
max score among faithfull cliques of 11 forms: 62.5
diviser Nombre de cli

In [49]:
newParadigmes,paradigmesSwim1,characteristicsSwim1,measuresSwim1=calculerResultats(swim1ContextParadigmes,"-Swim1")
%ding

/Users/gilles/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/gilles/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/gilles/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:545: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

UG 46321 OG 186 TP 15215 FP 5175
recall 0.228073331235 precision 0.739453732504
0.348620069426


# SWIM2

## Préparation des paradigmes

In [50]:
paradigmesOriginaux=paradigmes.copy()
paradigmesSample=paradigmesOriginaux[paradigmesOriginaux["lexeme"].isin(listeTest)]

In [51]:
paradigmes=newParadigmes.copy()
paradigmesColumns=paradigmes.columns.tolist()
for c in sampleCases:
    if not c in paradigmesColumns:
        print (c)
        paradigmes[c]=np.NaN

In [52]:
cliques=[]
cliquesScores={}
cliquesListes={}

formesScores={}
formesScoresNormes={}

print (datetime.datetime.now())
%time swim2ContextParadigmes=generateCliques(contextFree=True)

2018-01-02 12:31:25.559173


reprendre Nombre de cliques 712
max score among faithfull cliques of 17 forms: 35.0166715907
tâcher Nombre de cliques 389
max score among faithfull cliques of 18 forms: 98.9537513779
escamoter Nombre de cliques 520
no new faithfull clique, the previous one contained 11 forms
balayer Nombre de cliques 442
no new faithfull clique, the previous one contained 5 forms
surprendre Nombre de cliques 712
max score among faithfull cliques of 17 forms: 35.0166715907
rentrer Nombre de cliques 472
max score among faithfull cliques of 14 forms: 65.7582744948
pêcher Nombre de cliques 492
no new faithfull clique, the previous one contained 11 forms
dégotter Nombre de cliques 506
no new faithfull clique, the previous one contained 10 forms
assembler Nombre de cliques 320
max score among faithfull cliques of 17 forms: 103.172570001
amplifier Nombre de cliques 614
no new faithfull clique, the previous one contained 6 forms
punir Nombre de cliques 203
max score among faithfull cliques of 7 forms: 7.902071

chauffer Nombre de cliques 257
max score among faithfull cliques of 10 forms: 31.7085619679
tordre Nombre de cliques 250
max score among faithfull cliques of 9 forms: 20.5993728422
excommunier Nombre de cliques 647
max score among faithfull cliques of 11 forms: 28.6661374275
arracher Nombre de cliques 441
max score among faithfull cliques of 18 forms: 107.894350228
entasser Nombre de cliques 426
max score among faithfull cliques of 18 forms: 103.645279564
prévoir Nombre de cliques 308
max score among faithfull cliques of 16 forms: 70.5063251871
sucer Nombre de cliques 499
max score among faithfull cliques of 18 forms: 100.13396648
soupçonner Nombre de cliques 551
no new faithfull clique, the previous one contained 13 forms
tirer Nombre de cliques 424
max score among faithfull cliques of 19 forms: 116.905699153
pencher Nombre de cliques 471
max score among faithfull cliques of 18 forms: 107.894350228
troubler Nombre de cliques 413
max score among faithfull cliques of 17 forms: 94.158549

puiser Nombre de cliques 505
no new faithfull clique, the previous one contained 10 forms
partir Nombre de cliques 484
max score among faithfull cliques of 13 forms: 28.5375226821
armer Nombre de cliques 425
max score among faithfull cliques of 19 forms: 116.190661369
effrayer Nombre de cliques 350
max score among faithfull cliques of 7 forms: 11.3997556455
investir Nombre de cliques 201
max score among faithfull cliques of 17 forms: 47.6967931258
parier Nombre de cliques 337
max score among faithfull cliques of 10 forms: 36.5447697154
retirer Nombre de cliques 190
max score among faithfull cliques of 19 forms: 132.378521861
approcher Nombre de cliques 551
max score among faithfull cliques of 16 forms: 82.6806406372
composer Nombre de cliques 200
max score among faithfull cliques of 10 forms: 33.6151287157
unir Nombre de cliques 191
max score among faithfull cliques of 5 forms: 5.21381152798
acharner Nombre de cliques 424
max score among faithfull cliques of 19 forms: 117.698507127
mén

max score among faithfull cliques of 18 forms: 103.294088011
flotter Nombre de cliques 545
no new faithfull clique, the previous one contained 11 forms
procurer Nombre de cliques 386
max score among faithfull cliques of 19 forms: 116.798958469
nager Nombre de cliques 389
max score among faithfull cliques of 19 forms: 118.580695366
terminer Nombre de cliques 452
max score among faithfull cliques of 19 forms: 116.341428432
conduire Nombre de cliques 566
max score among faithfull cliques of 5 forms: 4.83476988195
suivre Nombre de cliques 580
max score among faithfull cliques of 11 forms: 19.5314492869
décarcasser Nombre de cliques 529
max score among faithfull cliques of 18 forms: 91.0856873777
bafouiller Nombre de cliques 657
no new faithfull clique, the previous one contained 8 forms
filer Nombre de cliques 39
max score among faithfull cliques of 15 forms: 96.1444818451
arroser Nombre de cliques 556
no new faithfull clique, the previous one contained 10 forms
glisser Nombre de cliques 4

...

max score among faithfull cliques of 11 forms: 28.6661374275
branler Nombre de cliques 451
max score among faithfull cliques of 14 forms: 63.4428514487
écosser Nombre de cliques 543
no new faithfull clique, the previous one contained 8 forms
ruisseler Nombre de cliques 254
max score among faithfull cliques of 15 forms: 77.2837933311
hésiter Nombre de cliques 462
max score among faithfull cliques of 18 forms: 102.918767717
écarter Nombre de cliques 221
max score among faithfull cliques of 18 forms: 104.869209138
pulluler Nombre de cliques 419
max score among faithfull cliques of 17 forms: 94.1743439647
confirmer Nombre de cliques 419
max score among faithfull cliques of 19 forms: 115.916693121
bosseler Nombre de cliques 254
max score among faithfull cliques of 15 forms: 77.2837933311
intéresser Nombre de cliques 536
no new faithfull clique, the previous one contained 10 forms
froisser Nombre de cliques 526
max score among faithfull cliques of 18 forms: 91.0531261889
régner Nombre de cli

max score among faithfull cliques of 19 forms: 122.033186507
détacher Nombre de cliques 79
max score among faithfull cliques of 9 forms: 32.2592205744
conspirer Nombre de cliques 453
max score among faithfull cliques of 19 forms: 114.23865484
sommer Nombre de cliques 549
no new faithfull clique, the previous one contained 13 forms
résister Nombre de cliques 398
max score among faithfull cliques of 18 forms: 101.874964046
parfaire Nombre de cliques 357
max score among faithfull cliques of 21 forms: 68.7201693691
réaliser Nombre de cliques 503
no new faithfull clique, the previous one contained 12 forms
protester Nombre de cliques 443
max score among faithfull cliques of 18 forms: 103.794923834
ôter Nombre de cliques 589
no new faithfull clique, the previous one contained 10 forms
rencontrer Nombre de cliques 248
max score among faithfull cliques of 7 forms: 16.8861509676
terrasser Nombre de cliques 536
max score among faithfull cliques of 18 forms: 85.0901662682
lasser Nombre de cliques

max score among faithfull cliques of 18 forms: 103.422739984
porter Nombre de cliques 369
max score among faithfull cliques of 18 forms: 106.557743395
descendre Nombre de cliques 509
max score among faithfull cliques of 15 forms: 39.9432446224
coller Nombre de cliques 275
max score among faithfull cliques of 15 forms: 76.1099728571
délivrer Nombre de cliques 384
max score among faithfull cliques of 19 forms: 112.737509442
cultiver Nombre de cliques 516
max score among faithfull cliques of 16 forms: 81.2773255852
écoeurer Nombre de cliques 651
no new faithfull clique, the previous one contained 10 forms
isoler Nombre de cliques 277
max score among faithfull cliques of 15 forms: 79.2076038979
accentuer Nombre de cliques 348
max score among faithfull cliques of 19 forms: 100.468349331
déchiffrer Nombre de cliques 391
max score among faithfull cliques of 19 forms: 117.43705305
dégainer Nombre de cliques 559
no new faithfull clique, the previous one contained 12 forms
déshonorer Nombre de c

max score among faithfull cliques of 19 forms: 118.850521386
guérir Nombre de cliques 362
max score among faithfull cliques of 8 forms: 15.513631993
daigner Nombre de cliques 647
no new faithfull clique, the previous one contained 11 forms
supprimer Nombre de cliques 439
max score among faithfull cliques of 19 forms: 117.085406457
embrasser Nombre de cliques 532
max score among faithfull cliques of 18 forms: 93.547856094
bousculer Nombre de cliques 409
max score among faithfull cliques of 17 forms: 98.3362459835
recoucher Nombre de cliques 543
max score among faithfull cliques of 17 forms: 92.1582033273
endiguer Nombre de cliques 412
max score among faithfull cliques of 19 forms: 121.813140798
diviser Nombre de cliques 497
no new faithfull clique, the previous one contained 11 forms
vernir Nombre de cliques 375
max score among faithfull cliques of 17 forms: 37.9173835892
informer Nombre de cliques 423
max score among faithfull cliques of 19 forms: 116.584071138
reconnaître Nombre de cl

max score among faithfull cliques of 17 forms: 37.9173835892
rechercher Nombre de cliques 479
max score among faithfull cliques of 18 forms: 104.415387219
transporter Nombre de cliques 412
max score among faithfull cliques of 18 forms: 104.00389685
décrire Nombre de cliques 348
max score among faithfull cliques of 10 forms: 36.3144746369
rêver Nombre de cliques 639
no new faithfull clique, the previous one contained 13 forms
réapparaître Nombre de cliques 605
no new faithfull clique, the previous one contained 11 forms
détendre Nombre de cliques 493
max score among faithfull cliques of 15 forms: 40.4560286882
constater Nombre de cliques 504
no new faithfull clique, the previous one contained 12 forms
régler Nombre de cliques 107
max score among faithfull cliques of 14 forms: 72.4637015949
râper Nombre de cliques 466
max score among faithfull cliques of 19 forms: 113.55945698
souder Nombre de cliques 527
no new faithfull clique, the previous one contained 13 forms
gonfler Nombre de cliq

In [53]:
newParadigmes,paradigmesSwim2,characteristicsSwim2,measuresSwim2=calculerResultats(swim2ContextParadigmes,"-Swim2")

UG 38928 OG 365 TP 22608 FP 8706
recall 0.321858717007 precision 0.713658890748
0.44363771941


In [54]:
nomFichierResultats=filePrefix+"-X-Resultats.yaml"
if os.path.isfile(nomFichierResultats):
    with open(nomFichierResultats, 'r') as stream:
            resultats=yaml.load(stream)
else:
    resultats={}

if casesType:
    sampleExt=casesType
else:
    sampleExt=sampleType
sampleId=sampleNumber.strip("-")+sampleExt
resultats[sampleId]={}
resultats[sampleId]["Swim1"]={}
resultats[sampleId]["Swim1"]["UG"]=characteristicsSwim1[0]
resultats[sampleId]["Swim1"]["OG"]=characteristicsSwim1[1]
resultats[sampleId]["Swim1"]["TP"]=characteristicsSwim1[2]
resultats[sampleId]["Swim1"]["FP"]=characteristicsSwim1[3]
resultats[sampleId]["Swim1"]["Precision"]=measuresSwim1[0]
resultats[sampleId]["Swim1"]["Recall"]=measuresSwim1[1]
resultats[sampleId]["Swim1"]["F-Measure"]=measuresSwim1[2]
resultats[sampleId]["Swim2"]={}
resultats[sampleId]["Swim2"]["UG"]=characteristicsSwim2[0]
resultats[sampleId]["Swim2"]["OG"]=characteristicsSwim2[1]
resultats[sampleId]["Swim2"]["TP"]=characteristicsSwim2[2]
resultats[sampleId]["Swim2"]["FP"]=characteristicsSwim2[3]
resultats[sampleId]["Swim2"]["Precision"]=measuresSwim2[0]
resultats[sampleId]["Swim2"]["Recall"]=measuresSwim2[1]
resultats[sampleId]["Swim2"]["F-Measure"]=measuresSwim2[2]

yaml.safe_dump(resultats, file(nomFichierResultats, 'w'), encoding='utf-8', allow_unicode=True)

In [55]:
%ding
%ding

In [56]:
print ("Sample",sampleNumber.strip("-"))
print ("Swim1",characteristicsSwim1,measuresSwim1)
print ("Swim2",characteristicsSwim2,measuresSwim2)

Sample 00-T10000-F3663
Swim1 (46321, 186, 15215, 5175) (0.739453732503888, 0.2280733312347289, 0.34862006942614593)
Swim2 (38928, 365, 22608, 8706) (0.713658890747814, 0.3218587170069189, 0.4436377194101314)


# Fin du traitement

In [73]:
paradigmesSwim2.columns.tolist()[25:]

[u'ai2S',
 u'fi2P',
 u'fi3P',
 u'fi3S',
 u'ii3P',
 u'ii3S',
 u'is1S',
 u'is3P',
 u'is3S',
 u'pI1P',
 u'pc1S',
 u'pc2S',
 u'pc3P',
 u'pc3S',
 u'pi3S',
 u'ppFP',
 u'ppFS',
 u'ppMP',
 u'ps2P',
 u'ps3P',
 u'ps3S']

In [76]:
paradigmesSwim2.pc1S

0          abEs6rE
1        abâdOn6rE
2             None
3          abOr6rE
4          abutirE
5         absât6rE
6        absOrb6rE
7        abstjêdrE
8          abim6rE
9        aksâty6rE
10       aksEpt6rE
11            None
12       akôpaJ6rE
13        akôplirE
14        akOrd6rE
15        akrOS6rE
16            None
17         akyz6rE
18      aksElEr6rE
19        aSarn6rE
20         aSEt6rE
21            None
22        aktiv6rE
23        adapt6rE
24         admEtrE
25        admir6rE
26        adOpt6rE
27         adOr6rE
28         adOs6rE
29        adrEs6rE
           ...    
1259        EmEtrE
1260          None
1261       Enôs6rE
1262      EparJ6rE
1263       EpEl6rE
1264      Epêgl6rE
1265          None
1266       EprâdrE
1267      Epruv6rE
1268      EpHiz6rE
1269      EtablirE
1270       EtaZ6rE
1271          None
1272        EtêdrE
1273        EtâdrE
1274       Etir6rE
1275       EtOn6rE
1276       Etuf6rE
1277     Etrâgl6rE
1278       EtydirE
1279          None
1280       E

# Calcul des distances

### Préparation des fichiers de référence
La comparaison se fait sur les lexèmes de *listeTest*
- initialForms : les formes initiales
- finalForms : les formes initiales plus les formes prédites
- goldForms : les formes de référence


#### Formes initiales

#### Formes avec prédictions

#### Formes de référence
- recoder les représentations phonologiques en fonction de l'échantillon

### Soustraction des formes initiales

### Évaluation des formes

#### Problèmes de génération
- sous-génération : formes de références sans prédiction
- sur-génération : formes prédites absentes de la référence

#### Formes comparables
- *predictedForms* => formes prédites présentes dans la référence
- *actualForms* => formes de référence présentes dans les prédictions

#### Préparation de la comparaison
- *compareForms* => tableau avec les formes à comparer

#### Comparaison des formes
- *sameForms* => formes identiques dans les prédictions et la référence
- *diffForms* => formes différentes dans les prédictions et la référence

#### Décompte du nombre de formes
À cause du traitement de la surabondance, le décompte des lignes ne tient pas compte du nombre de formes. Il faut compter les formes dans chaque ligne pour obtenir le nombre total.

#### Évaluation des différences
Certaines différences ne sont qu'apparentes à cause du traitement de la surabondance sous forme de chaine concaténée. Il faut transformer les surabondances en set() pour pouvoir faire une comparaison pertinente.

#### Calcul des grandeurs de mesure

### Sauvegarde des résultats pour analyse ultérieure

## Analyse des sur-générations

### Analyse par cases

Le facteur principal de **surgénération** est la production de **participes passés** (ppMP 662, ppFS 661, ppFP 661) pour les verbes qui n'ont qu'une seule forme de participe passé invariable (ppMS) : **54.78%** (1984/3622).

### Analyse par verbes

Le facteur principal de **surgénération** tient aux **verbes vestiges** et aux **météorologiques**.

## Analyse des sous-générations

### Analyse par cases

Le facteur principal de **sous-génération** est lié à la fréquence des TAM et de certaines personnes :
- TAM : 
 - imparfait du subjonctif 45.98% (7875/17126)
 - passé simple 12.70% (2176/17126)
- Personnes :
 - 2PL 46.11% (7897/17126)
 - 1PL 32.98% (5649/17126)


### Analyse par verbes

# Fin de l'analyse des résultats

### Test de la sauvegarde de l'analyse brute

## with open(analysisPrefix+"-analyseParadigmes.pkl","rb") as input:
    savedAnalyseParadigmes = pickle.load(input)    